<a href="https://colab.research.google.com/github/Sujal3141/Sentiment-Analysis-on-Product-Reviews/blob/main/Sentiment_Analysis_of_Product_Reviews_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing and Importing the required libraries

In [ ]:

import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install symspellpy



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 10.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import symspellpy


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
Women_clothing = pd.read_csv("/content/drive/MyDrive/BACKUP/Sentiment Analysis of Product Reviews: /Womens Clothing E-Commerce Reviews.csv")
amazon = pd.read_csv("/content/drive/MyDrive/BACKUP/Sentiment Analysis of Product Reviews: /amazon.csv")
apple = pd.read_csv("/content/drive/MyDrive/BACKUP/Sentiment Analysis of Product Reviews: /apple_iphone_11_reviews.csv")
flipkart = pd.read_csv("/content/drive/MyDrive/BACKUP/Sentiment Analysis of Product Reviews: /flipkart_product.csv",encoding = 'utf-16-le')

In [ ]:
amazon

,Text,label
0,This is the best apps acording to a bunch of ...,1
1,This is a pretty good version of the game for ...,1
2,this is a really . there are a bunch of levels...,1
3,"This is a silly game and can be frustrating, b...",1
4,This is a terrific game on any pad. Hrs of fun...,1
...,...,...
19991,this app is fricken stupid.it froze on the kin...,0
19992,Please add me!!!!! I need neighbors! Ginger101...,1
19993,love it! this game. is awesome. wish it had m...,1
19994,I love love love this app on my side of fashio...,1


In [ ]:
amazon.shape

(19996, 2)

In [ ]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19996 entries, 0 to 19995
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    19996 non-null  object
 1   label   19996 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


# Correcting the spelling of words in reviews using Textblob

In [ ]:

from textblob import TextBlob
def correcttext(text):
  return str(TextBlob(text).correct)

In [ ]:
text = [None]*19996
for i in range(19996):
  text[i] = correcttext(amazon["Text"][i])

In [ ]:
tokens = [None] * 19996
for i in range(19996):
  tokens[i] = word_tokenize(text[i])

In [ ]:
tokens = [[token.lower() for token in sentence] for sentence in tokens]

Removing Punctuations in tokens


In [ ]:
import string
tokens = [[token for token in sentence if token not in string.punctuation] for sentence in tokens]


Removing Stop Words

In [ ]:

stop_words = set(stopwords.words('english'))
tokens = [[token for token in sentence if token not in stop_words] for sentence in tokens]

Stemming

In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
tokens = [[stemmer.stem(token) for token in sentence] for sentence in tokens]

In [ ]:
tokens

In [ ]:
tokens = [[word for word in token_list if word.isalnum()] for token_list in tokens]


In [ ]:
texts = [' '.join(token_list) for token_list in tokens]

Converting into Vectors

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(texts)

Splitting into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,amazon["label"] , test_size=0.2, random_state=42)


Training using Logistic Regression

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = model.predict(X_test)


Displaying the accuracy of the trained model

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))


Accuracy: 0.8845
              precision    recall  f1-score   support

           0       0.86      0.63      0.73       978
           1       0.89      0.97      0.93      3022

    accuracy                           0.88      4000
   macro avg       0.87      0.80      0.83      4000
weighted avg       0.88      0.88      0.88      4000



Taking input from user and determing the sentiment

In [ ]:
y = (input("Enter your review : "))
t = vectorizer.transform([y])
pred = model.predict(t)
print(pred)
if pred==0:
  print("negative")
else :
  print("positive")


Enter your review : good
[1]
positive


In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump((model,vectorizer), file)

Extracting the requirement.txt file for getting the versions of modules and libraries use for the model deployment

In [ ]:
!pip freeze > requirements.txt